In [1]:
!nvidia-smi #GPUがあるかどうかの確認、あればTesla T4 の文字が示す

Sat Mar 15 12:03:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch #pytorch環境の確認（普段はあるはず）
torch.__version__

'2.6.0+cu124'

In [3]:
!pip3 install torchvision #実行環境の補足

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [4]:
pip install ultralytics #yolo環境のインストール

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.1/949.1 kB 26.0 MB/s eta 0:00:00


In [5]:
import os
!git clone https://github.com/ultralytics/yolov5  #YOLOソースコードをダウンロード
path = "yolov5"
os.chdir(path)

Cloning into 'yolov5'...
remote: Enumerating objects: 17274, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 17274 (delta 1), reused 0 (delta 0), pack-reused 17270 (from 3)
Receiving objects: 100% (17274/17274), 16.12 MiB | 30.17 MiB/s, done.
Resolving deltas: 100% (11856/11856), done.


In [6]:
!pip install -r requirements.txt #実行環境を確保（後はモデルを別途にダウンロードしてください）
#yoloのgithubページにreleaseをクリックし、show moreをクリックし、対応する分類モデルをダウンロードしてください
#今回はyolov5sを基づいて訓練をしたからyolov5-cls.ptをダウンロードし、ソースコードのフォルダにアップロードしてください

In [7]:
from google.colab import drive #Google driveに接続
drive.mount('/content/drive') #Google driveのメインフォルダ
path = "/content/drive/My Drive/yolo/data" #ここにデータセットの保存場所に変更してください
os.chdir(path) #データセットの位置に変更
os.listdir(path) #中身を確認

Mounted at /content/drive


['3', '0', '1', '2']

In [8]:
#生データは直接利用できないため、訓練データと検証データを分けて新しいフォルダに保存する

#!/usr/bin/env python
# -*- coding: utf-8 -*-
'''
@ Author: Rindon
@ Date: 2024-12-30 09:34:40
@ LastEditors: Rindon
@ LastEditTime: 2024-12-30 09:35:52
@ Description:
'''
import os
import random
import shutil

#SourceフィルダーとOutputフィルダーの定義
data_root = r"/content/drive/My Drive/yolo/data" #元のデータセットの保存場所（必要に応じて変更）
output_root = r"/content/data_ba" #分割したデータセットの保存場所

# Outputフィルダーを作る
folders = ['train', 'val']
for folder in folders:
    for label in ['0', '1', '2', '3']:
        os.makedirs(os.path.join(output_root, folder, label), exist_ok=True)

# 分割の比率 今は訓練8割、検証2割。必要に応じて変更できる
train_ratio = 0.8
val_ratio = 0.2

# 全ての種類にアクセス
for label in ['0', '1', '2', '3']:
    label_folder = os.path.join(data_root, label)
    images = [f for f in os.listdir(label_folder) if os.path.isfile(os.path.join(label_folder, f))]

    # データをランダムになる
    random.shuffle(images)

    # 数を計算する
    total_images = len(images)
    train_count = int(total_images * train_ratio)
    val_count = int(total_images * val_ratio)
    # 分割する
    train_images = images[:train_count]
    val_images = images[train_count:train_count + val_count]
    # 画像をコピーする
    for img in train_images:
        shutil.copy(os.path.join(label_folder, img), os.path.join(output_root, 'train', label, img))
    for img in val_images:
        shutil.copy(os.path.join(label_folder, img), os.path.join(output_root, 'val', label, img))

print("complete") #これを出力したら分割完了

complete


In [9]:
path = "/content/data_ba" #分割したフォルダに移動
os.chdir(path)
os.listdir(path)

#train と　val　が出ればOK

['train', 'val']

In [10]:
!python  "/content/yolov5/classify/train.py" --batch 128 --epochs 50 --data "/content/data_ba" --model "/content/yolov5/yolov5s-cls.pt" --device 0
#訓練結果をダウンロードしないと閉じて消す！

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-03-15 12:09:29.558482: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742040569.861260    4856 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742040569.939553    4856 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh